In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import queue
import threading
import re
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer
import sys
from sksurgerynditracker.nditracker import NDITracker
import os
import random
import multiprocessing

In [2]:
"""
   Initialize the NDI Aurora EM tracker
"""
SETTINGS_EM = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker_AURORA = NDITracker(SETTINGS_EM)
tracker_AURORA.start_tracking()

In [ ]:
"""
    Initialize the NDI Polaris Optical tracker (Track Butterfly Plate)
    MAKE SURE you have the Butterfly .rom file and its path
"""
SETTINGS_BF = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,

    # Windows Path
    "romfiles": ["PATH TO BUTTERFLY PLATE .ROM FILE"]
}
tracker_POLARIS = NDITracker(SETTINGS_BF)
tracker_POLARIS.start_tracking()

In [ ]:
BF_to_EN = np.eye(4)  # Fixed relationship, between Butterfly and Endoscope Coord. Systems, Using Yuan's Old Settings
EM_to_PI = np.eye(4)  # Fixed relationship, between EM Coil and Pin Coord. System.

PO_to_CT_calibration = np.load("PO_to_CT_calibration.npy")

CT_to_PO_calibration = np.linalg.inv(PO_to_CT_calibration)

FG_to_CT_calibration = np.load("FG_to_CT_calibration.npy")

CT_to_FG_calibration = np.linalg.inv(FG_to_CT_calibration)

PO_to_FG = np.dot(PO_to_CT_calibration, CT_to_FG_calibration)

In [ ]:
"""
    Initialize VTK Renderer
"""
renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(1000, 800) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.SetBackground(1, 1, 1) # set the background of the renderer

camera = vtk.vtkCamera()
renderer.SetActiveCamera(camera) # Initialize Camera

In [ ]:
"""
Import PIN .stl model to VTK
MAKE SURE you have the PIN .stl file and path
"""
pin_reader = vtk.vtkSTLReader() # read stl file
pin_reader.SetFileName("PATH TO PIN .STL FILE") # set the file name
pin_reader.Update() # update the reader

pin_mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
pin_mapper.SetInputData(pin_reader.GetOutput()) # set the input to the reader

pin_actor = vtk.vtkActor() # create an actor, an actor is used to display the data
pin_actor.SetMapper(pin_mapper) # set the mapper to the actor

renderer.AddActor(pin_actor) # add the actor to the renderer

In [ ]:
"""
    Get the PHANTOM/CT 3D model file names,
    Could be multiple models for different anatomical structures
    MAKE SURE you have the 3D model files (.stl or .vtk) of the PHANTOM/CT
"""
cwd = os.getcwd()
models = []

# For Windows Path
for file in os.listdir("PATH TO PHANTOM/CT .STL FOLDER"):
    if file.endswith('.vtk') or file.endswith('.stl'):
        models.append(file)

print(models)

In [ ]:
"""
    Loop through PHANTTOM/CT 3D model files, load them, and add them to the renderer
    MAKE SURE you have the 3D model files (.stl or .vtk) of the CT/Phantom
"""
for file in models:
    if file.endswith(".stl"):
        model_reader = vtk.vtkSTLReader()
        # Windows Path
        model_reader.SetFileName("PATH TO PHANTOM/CT 3D MODEL FOLDER" + file)

    elif file.endswith(".vtk"):
        model_reader = vtk.vtkPolyDataReader()
        # Windows Path
        model_reader.SetFileName("PATH TO PHANTOM/CT 3D MODEL FOLDER" + file)
    else:
        continue  # Skip unsupported formats
    # Update the reader to load the data
    model_reader.Update()

    # Create a mapper and actor to display the model
    model_mapper = vtk.vtkPolyDataMapper()
    model_mapper.SetInputData(model_reader.GetOutput())
    model_actor = vtk.vtkActor()
    model_actor.SetMapper(model_mapper)
    model_actor.GetProperty().SetColor(random.randint(0, 255)/255, random.randint(0, 255)/255, random.randint(0, 255)/255)

    # transform = vtk.vtkTransform()
    # transform.SetMatrix(transformation_matrix.flatten()) 
    # model_actor.SetUserTransform(transform)

    # if file.startswith("Box"):
    #     model_actor.GetProperty().SetOpacity(0.05)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    # if file.startswith("Box_1"):
    #     model_actor.GetProperty().SetOpacity(0.00)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    
    # Add the actor to the renderer
    renderer.AddActor(model_actor)

In [ ]:
def get_tracking_data(tracker):
    """Fetch latest tracking data"""
    port_handles, timestamps, framenumbers, transformation, quality = tracker.get_frame()
    transformation = np.array(transformation[0])
    return timestamps, transformation, quality

In [ ]:
data_lock = threading.Lock()
data_ready = threading.Event()
data_queue = queue.Queue()

def tracking_thread():
    global CT_to_EN, CT_to_PI
    while True:
        _, PO_to_BF, _ = get_tracking_data(tracker_POLARIS)
        _, FG_to_EM, _ = get_tracking_data(tracker_AURORA)

        PO_to_EN = np.dot(PO_to_BF, BF_to_EN)     # Update Endoscope Coord. System in Polaris
        CT_to_EN = np.dot(CT_to_PO_calibration, PO_to_EN)     # Update Endoscope in Phantom/CT Coord. System

        FG_to_PI = np.dot(FG_to_EM, EM_to_PI)
        CT_to_PI = np.dot(CT_to_FG_calibration, FG_to_PI)     # Update PIN in Phantom/CT Coord. System

        data_queue.put((CT_to_EN, CT_to_PI))

        time.sleep(1/40)

In [ ]:
def update_pose(obj, event):
    global CT_to_EN, CT_to_PI

    if not data_queue.empty():
        CT_to_EN, CT_to_PI = data_queue.get()

        # Update tool/PIN pose
        transform_PIN = vtk.vtkTransform()
        transform_PIN.SetMatrix(CT_to_PI.flatten())
        pin_actor.SetUserTransform(transform_PIN)

        # Update Endoscope Pose
        endo_matrix = CT_to_EN.flatten()
        camera.SetPosition(endo_matrix[3], endo_matrix[7], endo_matrix[11]) # get T(x, y, z)

        forward = np.array([endo_matrix[2], endo_matrix[6], endo_matrix[10]])  # Z 
        up = np.array([endo_matrix[1], endo_matrix[5], endo_matrix[9]])  # Y 

        focal_point = np.array(camera.GetPosition()) + forward  # focal point
        camera.SetFocalPoint(focal_point)  
        camera.SetViewUp(up)  

        # Update Renderer
        render_window.Render()
        print("Camera and tool pose updated")

In [ ]:
# 启动追踪线程
threading.Thread(target=tracking_thread, daemon=True).start()

# VTK 交互器
render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.AddObserver("TimerEvent", update_pose)
render_window_interactor.CreateRepeatingTimer(1)  # 1ms 刷新
render_window_interactor.Start()

In [ ]:
# while True:
#     # Read new data from Aurora
#     EM_handles, EM_timestamps, EM_framenumbers, EM_tracking, EM_quality = tracker_AURORA.get_frame()

#     # Read new data from Polaris
#     PO_handles, PO_timestamps, PO_framenumbers, PO_tracking, PO_quality = tracker_POLARIS.get_frame()

#     # Update EM data
#     FG_to_EM_updated = EM_tracking[0]

#     # Update Endoscope Butterfly in Polaris
#     PO_to_BF = PO_tracking[0]

#     # Update Endoscope Coord. System in Polaris
#     PO_to_EN = np.dot(PO_to_BF, BF_to_EN)

#     # Update Endoscope in Phantom/CT Coord. System
#     CT_to_EN = np.dot(CT_to_PO_calibration, PO_to_EN)

#     # Update PIN in Phantom/CT Coord. System
#     FG_to_PI = np.dot(FG_to_EM_updated, EM_to_PI)
#     CT_to_PI = np.dot(CT_to_FG_calibration, FG_to_PI)